앞서 \ref{chap:TyInf}장에서는 $\textbf{let}$식이라는 문법설탕에 해당하는 경우에만
지역변수(설탕은 녹인 핵심문법으로는 타입유추 가능한 인자에 적용된 함수의 파라미터)의
타입복원을 통해 타입유추의 기능을 확장하였다. 하지만, 재귀함수요약식에서
재귀함수 자신을 참조하는 변수의 타입 표기가 생략되면 복원하지 못하는 한계가 있었다.
이번 장에서는 재귀함수요약식의 타입복원이 어려운 이유를 알아보고,
이를 극복하기 위해 미지의 타입 정보를 부분적으로 포함하는 상태로 타입유추를 진행할 수
있도록 \'타입변수\'를 도입하여 수학에서 방정식을 푸는 방식을 타입복원과 타입유추 과정에 도입한다.
미지의 타입 정보를 나타내는 타입변수를 포함하는 방정식의 풀이는 \'일치화\'(unification) 문제를
해결하는 것에 해당하며, 일반적으로 타입 유추 과정에서는 여러 개의 타입변수에 대한 여러 개의 방정식을
만족하는 것이 타입규칙에 알맞은 타입이 존재하는 조건이 되므로 일치화 연립방정식을 풀어내야 한다.
정리하면, 이 장에서는 타입변수라는 개념을 도입하여 타입복원과 타입유추 과정에서 일치화 연립방정식이
함께 만들어지며, 이 과정에서 만들어진 연립방정식의 풀이를 통해 타입유추의 성공 여부가 결정된다.
이렇게 타입복원과 타입유추의 과정에서 타입에 대한 일치화 방정식을 구성하여 풀이하는 방식으로 접근하면
더 많은 타입 표기가 생략된 식에 대한 타입유추가 가능하다.

# 재귀함수요약식의 타입복원이 어려운 이유
\label{sec:WhyRecTyReconHard}
앞서 \ref{sec:recTyRecon}절에서 $\textbf{rec}\,f\,(\lambda x{:}\tau_2.e)$와 같은 재귀함수요약식에서 $f$의
타입 표기 복원이 어렵다고 언급한 바 있는데 여기서는 그 이유를 생각해 보자.
타입복원(`recon`) 함수는 타입환경(`TEnv`)과 식(`Expr`)을 인자로 받아 
타입유추(`infer`) 함수를 활용해 부분식으로부터 복원에 필요한 타입 정보를 얻어가며
생략된 타입 표기에 대한 복원을 진행한다.
설명의 편의를 위해 하스켈 코드와 타입규칙의 표현을 혼용한 의사코드(pseudocode)로 나타내자면
대략 다음과 같이 진행되어야 할텐데
\newline
`recon` $\Gamma$ $(\textbf{rec}\,f\,(\lambda x{:}\tau_2.e))$ = `[` $\!\!\textbf{rec}\,f{:}\tau_1\,(\lambda x{:}\tau_2.e)\!$ `|` $\!\tau_1$ $\leftarrow$ `infer` $\{x{:}\tau_2,\,f{:}??\}\Gamma$ $e\!\!$ `]` 
\newline
문제는 $??$의 위치가 과연 무슨 타입이어야 하는가이다.
$f$는 재귀함수요약식 자체를 대표하므로 당연히 $\tau_1$이면 되지 않냐고 생각했다면,
물론 맞는 생각이기는 하지만, 문제는 그런 방식으로 프로그램을 작성할 수가 없다는 것이다.
왜냐하면 $\tau_1$은 타입유추(`infer`)의 계산 결과로 비로소 알게 되는 타입인데,
`infer` 함수의 첫째 인자인 확장된 환경이 $\tau_1$을 포함한다면 이는 계산하기도 전에
결과값을 미리 알고 제공해야 한다는 이야기이므로 말이 되지 않는다.

이런 어려움을 극복하려면 말이 되지 않는 상황을 회피할 방법을 찾으면 된다.
말이 되지 않았던 이유는 $\tau_1$이라는 계산 결과를 미리 \`완전히\' 알고 타입환경에 $f$의 $\tau_1$에 대한 바인딩을 포함해야 한다는 생각 때문이었다.
그렇다면 불완전한 타입 정보를 타입 환경의 바인딩 내용으로 제공하거나 심지어 식에도 불완전한 타입 정보를 표기하는 것을 허용한다면 어떨까?
불완전한 정보 중에서 알려지지 않은 부분이 결국 무엇인지는 여전히 해결해야 할 숙제로 미뤄지긴 하겠지만, 최소한 완전히 말이 되지 않는 상황에서는 일단 벗어나서 생각해 볼 수 있다.
사실 이러한 접근 방식은 우리가 이미 수학에서 미지수를 나타내는 변수를 포함한 \`방정식\'의 풀이에서 익숙한 문제 해결 방식이다.
예컨대, $x = 2\times x - 3$이라는 방정식은 미지수 $x$를 포함한 우변의 산술식 $2\times x - 3$의 계산 결과가 좌변, 즉 $x$라는 말이다.
이렇게 아직 알려지지 않은 계산 결과를 일부분으로 포함하는 방정식을 해결하는 방법이 있듯, 재귀함수요약식의 타입복원과 타입유추도 마찬가지로 해결할 수 있으리라 희망해 볼 수 있다.

In [1]:
:opt no-lint
{-# LANGUAGE TypeSynonymInstances FlexibleInstances #-}

# 미지의 타입 정보를 나타내는 타입변수
\label{sec:UnknownTV}
미지의 타입 정보를 나타내는 타입변수를 FACT언어의 타입을 나타내는 데이터 타입 `Ty`에 다음과 같이 추가하였다.
타입변수는 하스켈 프로그램에서 `TV i`로 작성하며 (여기서 `i::Int`) 보기좋게 노트북에 디스플레이할 때는 $t_i$로 표시되도록 하였다.
앞서 다룬 예시인 $\textbf{rec}\,f\,(\lambda x{:}\mathbb{Z}.e)$의 타입을 복원하려 할 때 타입변수를 활용해 생각을 한번 펼쳐 보자.
전체 재귀함수를 대표하는 $f$는 정수($\mathbb{Z}$) 타입의 인자를 받는다는 것은 알지만 결과값의 타입이 무엇이어야 하는지는 아직 잘 모른다.
따라서 $f$의 타입을 $\mathbb{Z}{\to}t_i$(하스켈 코드로는 `Arr Z (TV i)`)로 바인딩해
확장된 타입환경 $\{f{:}\mathbb{Z}{\to}t_i\}\Gamma$에서 $e$의 타입으로 유추된 결과 $\tau_1$과
$t_i$를 일치화(unify)하는 방정식을 구성하여 그 해를 구할 수 있다면 알맞은 타입을 찾은 것이다.
그러나, 방정식을 구성했더라도 해가 존재하지 않으면 알맞은 타입이 없다는 말이다.

In [2]:
type Nm = String  -- 변수 이름은 문자열로

data Ty = TV Int | Z | Arr Ty Ty  deriving (Eq,Ord)
data Expr = Var Nm                      -- x
          | Lam Nm (Maybe Ty) Expr      -- (\x:t.e)
          | Rec Nm (Maybe Ty) Expr      -- rec f:t (\x:t2.e)
          | App Expr Expr               -- (e1 e2)
          | Lit Int                     -- n
          | Add Expr Expr               -- e1 + e2
          | If Expr Expr Expr           -- if e then e1 else e0
          deriving (Eq, Ord)
          
data Val = Cl Expr Env | Vi Int  deriving (Eq,Ord)
type Env = [ (Nm, Val) ]
type TEnv = [ (Nm, Ty) ]

In [3]:
instance Show Ty where
  showsPrec p (TV i) = showString("t_{"++show i++"}")
  showsPrec _ Z = showString "\\mathbb{Z}"
  showsPrec p (Arr t1 t2) = showParen (p > 1) $
      showsPrec p t1 . showString "{\\to}" . showsPrec p t2
instance Show Expr where -- LaTeX 소스코드 생성하는 Show 인스턴스 직접 선언
  showsPrec _ (Var x) = showString x
  showsPrec p (Lam x mt e) = showParen (p > 1) $
      showString ("\\lambda "++x) . showsMT mt .
      showString "." . showsPrec 1 e
  showsPrec p (Rec f mt e) = showParen (p > 9) $
      showString ("\\textbf{rec}\\,"++f) . showsMT mt .
      showString "\\, " . showsPrec 2 e
  showsPrec p (App e1 e2) = showParen (p > 9) $
      showsPrec 9 e1 . showString "\\;" . showsPrec 10 e2
  showsPrec _ (Lit n) = shows n
  showsPrec p (Add e1 e2) = showParen (p > 6) $
      showsPrec 6 e1 . showString "+" . showsPrec 7 e2
  showsPrec p (If e e1 e0) = showParen (p > 2) $
      showString "\\textbf{if}\\;" . showsPrec 3 e .
      showString "\\;\\textbf{then}\\;" . showsPrec 3 e1 .
      showString "\\;\\textbf{else}\\;" . showsPrec 3 e0

showsMT (Just t) = showString "{:}" . showsPrec 0 t
showsMT Nothing  = id 

import IHaskell.Display (latex)
dTeX e = display [latex $ "$"++show e++"$"]
instance IHaskellDisplay Expr where display = dTeX
instance IHaskellDisplay Ty   where display = dTeX

In [4]:
import Data.List (intercalate)
instance {-# OVERLAPS #-} Show Env where
    show env = "\\{"++ intercalate ",\\," (map show env) ++"\\}"
instance {-# OVERLAPS #-} Show TEnv where
    show tenv = "\\{"++ intercalate ",\\," (map show tenv) ++"\\}"
instance {-# OVERLAPS #-} Show (Nm,Val) where
    show (x,v) = x++"{\\mapsto}"++show v
instance {-# OVERLAPS #-} Show (Nm,Ty) where
    show (x,t) = x++"{\\mapsto}"++show t
instance Show Val where
    show (Cl e env) = "\\langle "++show e++","++show env++"\\rangle "
    show (Vi n) = " "++show n++" "
instance {-# OVERLAPS #-} IHaskellDisplay Env    where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay TEnv   where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Val) where display=dTeX
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Ty)  where display=dTeX
instance IHaskellDisplay Val                     where display = dTeX

# 다소 무책임한 타입복원(`recon`)
지금까지는 기존의 FACT언어에 대한 타입복원과 타입유추에 대한 이해를 바탕으로 설명하기 위해
\ref{chap:TyInf}장에서 타입복원(`recon`) 함수에 타입유추(`infer`) 함수를 활용하던 흐름에 맞춰
왜 재귀함수타입요약식의 타입복원 및 타입유추가 어려운지 (\ref{sec:WhyRecTyReconHard}절) 이야기하였다.
그런데, \ref{sec:UnknownTV}절에서 언급했듯이 결국 타입복원 및 타입유추가 최종적으로 성공했는지는
방정식의 해를 찾을 수 있는지에 달려 있다.
그러니까 타입복원 과정에서 더 많은 일을 하면 조금 더 풀기 쉬운 형태로 방정식을 만들어낼지는 모르지만
일반적으로 알맞은 타입이 있는지까지 결론내지는 못한다. 그러니, 타입유추 등 다른 부분에서 필요한
최소한으로, 어찌 보면 조금 무책임하게 방정식을 푸는 쪽에 할 일을 몰아서 미루는 방식으로, `recon`을 작성하자.
아직 불확실한 정보를 타입변수로 나타낼 수 있음을 활용해 타입이 생략되어 `Nothing`이 나타난 곳을
서로 겹치지 않는 타입변수로 타입을 표기하여 `Just (TV i)`로 복원하기만 하면 된다.
원리는 간단한데, 서로 겹치지 않도록 `i`를 계산하는 내용 때문에 `rcn`의 코드가 조금 길어졌다.
이렇게 되면, `recon`이 무조건 성공하므로 더 이상 `[Expr]`이 아닌 그냥 `Expr`을
함수의 결과값 타입으로 삼는다.

In [5]:
recon :: TEnv -> Expr -> Expr  -- 각각의 Nothing을 고유한 타입변수로
recon tenv e = rcn 0 tenv e

rcn i _    e@(Var _)    = e
rcn i tenv (Lam x Nothing  e) = Lam x (Just t) (rcn (1+i) tenv' e)
                            where tenv' = (x,t):(tenv)
                                  t = TV i
rcn i tenv (Lam x (Just t) e) = Lam x (Just t) (rcn i tenv' e)
                            where tenv' = (x,t):tenv
rcn i tenv (Rec f Nothing  e) = Rec f (Just t) (rcn (1+1) tenv' e)
                            where tenv' =  (f,t):(tenv)
                                  t = TV i
rcn i tenv (Rec f (Just t) e) = Rec f (Just t) (rcn (1+i) tenv' e)
                            where tenv' = (f,t):tenv
rcn i tenv (App e1 e2)  = App e1' e2'   where e1' = rcn i tenv e1
                                              e2' = rcn j tenv e2
                                              j = 1 + maxTV i e1'
rcn i _    e@(Lit _)    = e
rcn i tenv (Add e1 e2)  = Add e1' e2'   where e1' = rcn i tenv e1
                                              e2' = rcn j tenv e2
                                              j = 1 + maxTV i e1'
rcn i tenv (If e e1 e0) = If e' e1' e0' where e'  = rcn i tenv e
                                              e1' = rcn j tenv e1
                                              e0' = rcn k tenv e0
                                              j = 1 + maxTV i e'
                                              k = 1 + maxTV j e1'
-- maxTV i e는 i를 포함해 e에 나타나는 모든 타입변수 정수값들의 최대값 
maxTV :: Int -> Expr -> Int
maxTV i e = maximum (i : concatMap collectTV (collectTy e))

collectTy :: Expr -> [Ty]  -- 식에 나타나는 모든 타입 표기로 이루어진 리스트
collectTy (Var _)      = []
collectTy (Lam x Nothing  e) = collectTy e
collectTy (Lam x (Just t) e) = t : collectTy e
collectTy (Rec x Nothing  e) = collectTy e
collectTy (Rec x (Just t) e) = t : collectTy e
collectTy (App e1 e2)  = collectTy e1 ++ collectTy e2
collectTy (Lit _)      = []
collectTy (Add e1 e2)  = collectTy e1 ++ collectTy e2
collectTy (If e e1 e0) = collectTy e ++ collectTy e1 ++ collectTy e0

collectTV :: Ty -> [Int]  -- 타입에 나타나는 모든 타입변수 정수값들의 리스트
collectTV (TV i)      = [i]
collectTV Z           = []
collectTV (Arr t1 t2) = collectTV t1 ++ collectTV t2

In [6]:
e1 = Lam "x" Nothing . Lam "x'" Nothing $ Var "x" `Add` Var "x'"  
e2 = Lit 2 `Add` Var "y"  -- y는 자유변수!!!
e5 = Lam "z" Nothing (Var "z") `App` Lit 5
App (App e1 e2) e5

\noindent
위 식에서는 하스켈 코드에서 `Nothing`으로 표기된 세 군데에 타입 표기가 생략되어 있다.
이를 아래와 같이 이 장에서 새로 작성한 `recon` 함수로 타입복원하면 각각의 위치에
겹치지 않는 타입변수 $t_0$, $t_1$, $t_2$가 표기된 식이 만들어진다.

In [7]:
recon [("y",Z)] (App (App e1 e2) e5)

\noindent
참고로 타입환경(`[("y",Z)]`)는 이 장에서 새로 작성한 `recon`에서 활용하지 않으므로 (다른 추가 기능을 구현하지 않는 한)
사실상 필요없는 인자이지만 \ref{chap:TyInf}장과 비슷한 형식으로 일관성을 유지하기 위해 남겨놓았다.

# 풀어야 할 연립방정식을 함께 만들어내는 타입유추 과정
이제 타입유추 함수(`infer`)는 타입(`Ty`)과 함께 풀어야 할 방정식(`Eqn`)도 함께 만들어내야 한다.
방정식의 해가 존재하는가의 여부에 따라 알맞은 타입의 존재 여부가 결정되므로,
앞서 `recon`을 항상 결과값으로 식(`Expr`)을 만들어내는 데 성공하는 것과 마찬가지 논리로
`infer`도 항상 결과값으로 타입(`Ty`)을 만들어내는 데 거의 항상 성공하는 것으로 취급할 수 있다.
단, 이름의 유효범위가 잘못되었거나 타입복원을 제대로 수행하지 않은 식에 대해서는
방정식을 굳이 풀어 볼 필요도 없이 너무나 명확히 실패하는 경우이므로  `error`로 비정상 종료 처리한다.
따라서 기존에 실패하는 경우를 고려한 `[Ty]` 대신 그냥 `Ty`를 결과값으로 계산하면 되는데,
이와 함께 연립방정식(`[Eqn]`)도 만들어내야 하므로 `infer`의 결과값은
이 두 정보의 순서쌍 타입인 `(Ty, [Eqn])`으로 표현된다.

In [8]:
type Eqn = (Ty,Ty) -- 방정식의 의미는 두 타입이 일치화(unify)될 수 있는지

infer :: Int -> TEnv -> Expr -> (Ty,[Eqn])
infer i tenv (Var x) =
  case lookup x tenv of
    Just t  -> (t, [])
    Nothing -> error( x ++ " not found in " ++ show tenv )
infer i tenv (Lam x (Just t2) e) = (Arr t2 t, eqns)
  where (t,eqns) = infer i ((x,t2):tenv) e
infer i tenv (Lam x Nothing   e) =
  error( "missing type annot. in " ++ show(Lam x Nothing e) )
infer i tenv (Rec f (Just t) e)  = undefined -- 적절히 작성
infer i tenv (Rec f Nothing  e)  =
  error( "missing type annot. in " ++ show(Rec f Nothing e) )
infer i tenv (App e1 e2)  = undefined -- 적절히 작성
infer i tevv (Lit _)      = (Z, [])
infer i tenv (Add e1 e2)  = (Z, eqns1 ++ eqns2)
  where eqns1 = check i tenv e1 Z
        eqns2 = check j tenv e2 Z
        j = 1 + maximum (maxTV i e1 : collectTVeqns eqns1)
infer i tenv (If e e1 e0) = undefined -- 적절히 작성

collectTVeqns eqns = [i | (t',t) <- eqns,
                          i <- collectTV t' ++ collectTV t]

check :: Int -> TEnv -> Expr -> Ty -> [Eqn]
check i tenv e t = (t',t) : eqns  where (t',eqns) = infer i tenv e

\noindent
타입검사 함수(`check`)를 타입유추 함수(`infer`)로 간단히 정의하는 것은 마찬가지지만,
당장 유추된 타입(`t'`)과 제시된 타입(`t`)이 같은가 판단하기에는 정보가 불완전할지 모르므로 일단 보류하고,
두 타입의 일치화 방정식을 구성해 타입유추로부터 얻은 연립방정식(`eqns`)에 추가한다.
따라서, `check`의 결과값은 더 이상 `Bool`이 아닌 `[Eqn]` 타입이 된다.
이 장의 `infer`와 `check`에 추가된 파라미터 `i`는 `[6]`번 셀 `rcn` 함수의 `i`와 마찬가지로
기존의 타입변수와 겹치지 않도록 어디서부터 시작할지 알려주는 정수 인자를 받는다.

In [9]:
instance {-# OVERLAPS #-} Show [Eqn] where
    show eqns = "\\{"++ intercalate ",\\," (map show eqns) ++"\\}"
instance {-# OVERLAPS #-} Show Eqn where
    show (t',t)= show t'++"\\,\\overset{_?}{=}\\,"++show t

instance {-# OVERLAPS #-} IHaskellDisplay Eqn   where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay [Eqn] where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay (Ty,[Eqn]) 
                                                where display = dTeX

In [10]:
Var "x" `Add` Var "x'" -- e1의 함수 몸체 부분에 대한 예시
infer 2 [("x'",TV 1),("x",TV 0)] (Var "x" `Add` Var "x'")

\noindent
위의 연립방정식에 만족하는 해($\{t_0{\mapsto}\mathbb{Z},t_1{\mapsto}\mathbb{Z}\}$)가 존재하므로
$x+x'$에 대해 주어진 타입환경에서 타입규칙에 맞는 타입 $\mathbb{Z}$을 찾을 수 있다고 판단한다.
하지만 아래와 같은 경우처럼, 똑같은 식이라도 주어지는 타입환경에 따라서는 연립방정식을 만족하는 해가 존재하지 않는다.

In [11]:
Var "x" `Add` Var "x'" -- 잘못된 타입환경이 주어진 경우의 예시
infer 2 [("x'",Arr Z Z),("x",TV 0)] (Var "x" `Add` Var "x'")

\noindent
연립방정식을 만족시키려면 포함하는 모든 방정식을 참으로 만드는 타입 치환을 찾아야 한다. 그런데,
$\mathbb{Z}{\to}\mathbb{Z}\,\overset{?}{=}\,\mathbb{Z}$는 애초부터 어떤 타입 치환을 적용해도
거짓이므로 이와 같은 방정식이 포함되면 해가 존재할 수 없다. 이처럼 `infer` 함수가
에러 없이 실행되어 만들어내는 순서쌍의 첫 요소로  구체적인 타입이 계산되더라도, 연립방정식의
풀이 결과에 따라 타입유추가 최종적으로 실패했는지 성공했는지 판단됨에 유의해야 한다.

In [12]:
e1  -- e1은 타입 표기가 생략된 곳이 있다
infer 0 [] e1 -- 그래서 infer는 에러남

: 

In [13]:
e1' = recon [] e1 
e1'         -- e1의 타입을 복원한 e1'

In [14]:
maxTV 0 e1' -- e1'의 최대 타입변수 정수값

1

In [15]:
infer (maxTV 0 e1' + 1) [] e1' -- 타입을 복원하여 infer하면 에러없이 실행됨

In [16]:
infer' e = infer (maxTV 0 e + 1) [] e -- 닫힌식 e에 대한 타입유추 함수

\noindent
아직 `[8]`번 셀에서 `undefined`로 남겨져 있는 부분이 있기 때문에 일반적으로 `Var`, `Lam`, `Add` 외의 다른 문법요소가 포함된 식에 대해서는 타입유추(`infer`)가 제대로 동작하지 않는다.
연습문제로 `[8]`번 셀의 `infer` 함수를 완성하여 다른 문법요소가 포함된 식에 대한 타입유추도 `infer` 또는 `infer'`로 테스트해 보라.

# 일치화 방정식의 풀이
\label{sec:SolveUnify}
우리가 다루는 타입유추의 맥락에서 타입(`Ty`)에 대한 일치화를 설명하자면,
일치화 방정식이란 $\mathbb{Z}{\to}t_0\overset{_?}{=}t_1{\to}\mathbb{Z}{\to}\mathbb{Z}$와 같이
왼쪽과 오른쪽에 모두 변수($t_i$)가 나타날 수 있는 방정식을 말하며, 양쪽 항 모두 또는 한쪽 항에만
적용했을 때 똑같게 만드는 치환(substitution)을 찾으면 방정식을 푼 것으로 본다. 방금 예시로 든 방정식에는
$S=\{t_0{\mapsto}\mathbb{Z}{\to}\mathbb{Z},\,t_1{\mapsto}\mathbb{Z}{\to}\mathbb{Z}\}$라는
해가 존재한다. 타입 치환 $S$를 왼항에 적용하여 $t_0$을 $\mathbb{Z}{\to}\mathbb{Z}$로 바꿔치면
$S(\mathbb{Z}{\to}t_0)=\mathbb{Z}{\to}\mathbb{Z}{\to}\mathbb{Z}$가 되며, 마찬가지로 치환 $S$를 오른항에 적용하여
$t_1$을 $\mathbb{Z}$로 바꿔치면 $S(t_1{\to}\mathbb{Z}{\to}\mathbb{Z})=\mathbb{Z}{\to}\mathbb{Z}{\to}\mathbb{Z}$가 되어 똑같아진다.
즉, 타입에 대한 일치화 방정식 $\mathbb{Z}{\to}t_0\overset{_?}{=}t_1{\to}\mathbb{Z}{\to}\mathbb{Z}$를
만족하는 타입 치환 $S$를 찾았으므로 방정식을 푼 것이다.

방정식의 해를 나타내기 위한 타입 치환(type substitution, `TSubs`)을 하스켈 구현에서는
\ref{sec:CESK}절에서 CESK기계를 작성하며 활용한 `HashMap` 데이터 구조로 아래와 같이 선언한다.
또한, 아무것도 바꿔치지 않는 빈 타입 치환(`emptyTSub`)을 라이브러리에서 제공되는 빈 `HashMap`으로써 정의한다.

In [17]:
import qualified Data.HashMap.Strict as M -- M.funcName으로 활용
import Data.HashMap.Strict (HashMap, (!), (!?)) -- 앞에 M. 없이
type TSubs = HashMap Int Ty -- 타입변수의 정수값을 타입에 대응시키는 타입 치환
emptyTSubs = M.empty :: TSubs -- 빈 타입 치환

In [18]:
instance {-# OVERLAPS #-} Show TSubs where
  show tsub = "\\{"++ intercalate ",\\," (map show $ M.toList tsub)
            ++"\\}"
instance {-# OVERLAPS #-} Show (Int,Ty) where
  show (i,t) = "t_"++show i++"{\\mapsto}"++show t
instance {-# OVERLAPS #-} IHaskellDisplay TSubs  where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay (Int,Ty) where display=dTeX

In [19]:
tyL = Arr Z      (TV 0) 
tyR = Arr (TV 1) (Arr Z Z)
(tyL,tyR) -- 일치화 방정식

\noindent
앞서 예시로 살펴본 일치화 방정식을 하스켈 코드로 옮기면 위와 같다.
이 방정식의 해를 나타내는 타입 치환은 하스켈 코드로 다음과 같이 작성할 수 있다.

In [20]:
tsubs01 = M.fromList [(0,Arr Z Z),(1,Z)] :: TSubs
tsubs01 -- 타입 치환

\noindent
위의 타입 치환(`tsubs01`)이 방정식의 해인지 검산하기 위해
타입 치환($S$)을 타입($\tau$)에 적용하는 하스켈 함수 `applyTSubs`를
아래와 같이 작성하였다. 타입 치환을 타입에 적용하는 $S(\tau)$를
하스켈 코드로는 대략 `applySubst` $S$ $\tau$로 옮길 수 있다.

In [21]:
applyTSubs :: TSubs -> Ty -> Ty
applyTSubs tsubs (TV i) = case tsubs !? i of Nothing -> TV i
                                             Just t  -> t
applyTSubs tsubs Z           = Z
applyTSubs tsubs (Arr ta tb) = Arr ta' tb'
                             where ta' = applyTSubs tsubs ta
                                   tb' = applyTSubs tsubs tb

\noindent
이제, 실제로 `tsubs01`이 방정식의 해인지 하스켈 프로램으로 검산해 보자.
위 `applyTSubs` 함수를 활용해 `tsubs01`를 방정식 왼쪽의 타입(`tyL`)과
오른쪽의 타입(`tyR`)에 각각 적용하면 다음과 같이 그 결과를 확인해 볼 수 있다.

In [22]:
applyTSubs tsubs01 tyL 
applyTSubs tsubs01 tyR

연립방정식을 만족시키는 치환이라고 해서 모두 말이 되지 않음에 유의해야 한다.
예컨대, $t_0 \overset{?}{=} \mathbb{Z}{\to}t_0$는 만족시킬 수 없는 방정식임이 분명하다.
왜냐하면, 오른항은 $t_0$를 포함하는 $t_0$보다 더 큰 구조인데 왼항은 $t_0$로만 이루어져 있으므로 상식적으로 일치화될 수 없다.
그런데 $S'=\{t_0{\,\mapsto,}\mathbb{Z}{\to}t_0\}$라는 치환을 방정식의 왼항에 적용하면 $S(t_0)=\mathbb{Z}{\to}t_0$로 오른항과 똑같아져 일치화되는 것처럼 보인다.
이 치환의 문제점은 적용 회수에 따라 다른 결과가 된다는 점이다. 이를테면 $S'$를 왼항에 두 번 적용하면
$S'(S'(t_0)) = S'(\mathbb{Z}{\to}t_0) = \mathbb{Z}{\to}\mathbb{Z}{\to}t_0$로 한 번만 적용했을 때와 다르다.
참고로, 유한한 구조를 일치시키는 말이 되는 치환 $S$는 1회 이상 몇 번 적용하든 결과가 같은 \`멱등성'(idempotence)을 지녀야 한다
(즉, $S(\tau) = S(S(\tau)) = S(S(S(\tau))) = \cdots$). 방금 살펴본 $S'$가 멱등성을 만족하지 못하는 이유를 생각해 보면,
치환에 속하는 대응 $t_0{\,\mapsto\,}\mathbb{Z}{\to}t_0$에서 치환하려는 변수($t_0$)가 치환 결과($\mathbb{Z}{\to}t_0$)에 나타나기(`occurs`)
때문에 한 번 적용한 다음에도 계속 같은 치환을 적용했을 때 바꿔칠 거리가 남아있기 때문이다.
따라서, 타입변수를 나타내는 정수값(`i`)이 특정 타입(`Ty`)에 나타나는지 검사하는 다음과 같은 `occurs` 함수로 하스켈 코드에서 말이 되지 않는 타입 치환을 걸러낼 수 있다.

In [23]:
occurs :: Int -> Ty -> Bool
occurs i (TV j)      = i == j
occurs i Z           = False
occurs i (Arr ta tb) = occurs i ta || occurs i tb

\begin{align*}
\text{타입}~\tau \in \texttt{Ty} \;::=\; t_i \,\mid\, \mathbb{Z} \,\mid\, \tau{\to}\tau
\quad
&&\text{연립방정식}~&
P ::= \{ \tau_1\overset{_?}{=}\tau_1',\,\ldots,\,\tau_n\overset{_?}{=}\tau_n'\}
\\
&&\text{타입 치환}~&
S ::= \{\,t_{i_1}{\mapsto}\tau_1,\,\ldots,t_{i_m}{\mapsto}\tau_m \}
\\[-3em]
\end{align*}
$~$\fbox{$P;S \xmapsto{~_\mathcal{U}~} P';S'$}\hfill$\phantom{A}$\vspace*{-1ex}
\begin{align*}
\text{(항등식 제거)}\qquad
\{\tau\overset{_?}{=}\tau\}\uplus P;S &\xmapsto{~_\mathcal{U}~} P;S
\\
\text{(분해)}~~
\{\tau_1{\to}\tau_2\overset{_?}{=}\tau_1'{\to}\tau_2'\}\uplus P;S &\xmapsto{~_\mathcal{U}~}
 \{\tau_1\overset{_?}{=}\tau_1',\,\tau_2\overset{_?}{=}\tau_2'\}\uplus P;S
\\
\text{(변수 제거)}\qquad\quad
\{t_i\overset{_?}{=}\tau\}\uplus P;S &\xmapsto{~_\mathcal{U}~} \{t_i{\mapsto}\tau\}(P);\{t_i{\mapsto}\tau\}\uplus \{t_i{\mapsto}\tau\}(S)\\
&\qquad\quad~\text{(단,  $t_i \neq \tau$이고 $t_i$가 $\tau$에 나타나지 않음)}
\\
\text{(좌우 반전)}\qquad\quad
\{\tau\overset{_?}{=}t_j\}\uplus P;S &\xmapsto{~_\mathcal{U}~}
 \{t_j\overset{_?}{=}\tau\}\uplus P;S
\quad\text{(단, $\tau$는 타입 변수 아님)}
\\[-3em]
\end{align*}
\begin{align*}
S(P) &\;= \big\{S(\tau)\overset{_?}{=}S(\tau') \;\big|\; \tau\overset{_?}{=}\tau' \in P\big\}
&&\text{\scriptsize연립방정식에 대한 타입 치환 적용}
\\
\{t_i{\mapsto}\tau\}(S) &\;= \big\{t_k{\mapsto}\{t_i{\mapsto}\tau\}(\tau_k) \;\big|\; t_k{\mapsto}\tau_k \in S\big\}
&&\text{\scriptsize타입 치환에 대한 한 개의 타입변수 치환}
\end{align*}
\caption{타입($\texttt{Ty}$)에 대한 일치화 연립방정식의 규칙 기반 풀이 방식 $\mathcal{U}$
         \label{fig:RuleBasedUforTy} }

일반적으로 일치화 방정식을 활용하는 상황에서는 여러 개의 일치화 방정식을 동시에 만족시켜야 하는
연립방정식을 다루는 경우가 많다. 또한, 하나의 복잡한 방정식의 풀이를 여러 개의 더 단순한 형태로
분해된 연립방정식의 풀이로 접근할 수도 있다.
일차항(first-order term)에 대한 문법적 일치화(syntactic unifcation) 연립방정식의
\`\`규칙 기반 풀이 방식 $\mathcal{U}$\'\'(rule-based appraoch $\mathcal{U}$)\cite{BaaderSnyder01}를
우리가 다루는 타입(`Ty`)에 맞춰 정리하면 그림 \ref{fig:RuleBasedUforTy}과 같다. 참고로,
방정식을 풀기 위한 $P;S \xmapsto{~_\mathcal{U}~} P';S'$ 형태의 규칙을
일치화 연립방정식이라는 형식언어에 의미를 부여하는 작은걸음 의미구조로 이해해도 좋다.
더 이상 풀 방정식이 없는 경우에 정상 종료되며 연립방정식의 풀이에 성공한다.

In [24]:
solve :: [Eqn] -> [TSubs] -- 해가 없으면 빈 리스트
solve eqns = sol eqns emptyTSubs

sol :: [Eqn] -> TSubs -> [TSubs]
sol []                   tsubs = [tsubs] -- 더 처리할 방정식 없음
sol ((Z, Z)      : eqns) tsubs        = sol eqns tsubs -- 항등식 제거
sol ((TV i, TV j): eqns) tsubs | i==j = sol eqns tsubs -- 항등식 제거
sol ((Arr t1 t2,      -- 분해 (함수의 정의역끼리 공역끼리)
      Arr s1 s2) : eqns) tsubs = sol ((t1,s1):(t2,s2):eqns) tsubs
sol ((Z, Arr _ _): eqns) tsubs = [] -- 정수와 함수는 일치화 불가능
sol ((Arr _ _, Z): eqns) tsubs = [] -- 정수와 함수는 일치화 불가능
sol ((TV i, t) : eqns) tsubs
    | occurs i t = []  -- 부분이 전체를 포함하므로 해결 불가능
    | otherwise  = sol eqns' (M.insert i t tsubs') -- 변수 제거
    where eqns' = [ (i2t t1, i2t t2) | (t1,t2) <- eqns ]
          tsubs' = M.map i2t tsubs
          i2t = applyTSubs (M.insert i t emptyTSubs)
sol ((t, TV j) : eqns) tsubs = sol ((TV j,t):eqns) tsubs -- 좌우 반전

위의 `solve`는 방정식 풀이의 작은걸음($\xmapsto{~_\mathcal{U}~}$)에 대응되는 큰걸음 의미구조에 해당한다.
즉, 작은걸음 규칙($\xmapsto{~_\mathcal{U}~}$)을 반복적으로 적용해 정상 종료조건으로 성공하며
만들어진 타입치환이 포함된 리스트(`[TSubs]`)를 계산하거나, 방정식의 해가 없어 비정상 종료로 실패하는
경우에는 빈 리스트를(`[]`)를 계산한다.
`solve`를 구현하는 `sol`은 꼬리재귀(tail recursion)로 작은걸음 규칙을 반복적으로 적용하는 내용으로 작성되어 있다.
`sol`에서 더 이상 풀이할 방정식이 없는 정상 종료와 `[]`를 결과로 계산하는 비정상 종료를
처리하는 부분을 제외하면 그림 \ref{fig:RuleBasedUforTy}의 규칙을 차례로 하스켈 코드로 옮긴 것이다.
규칙 하나만 예를 들자면, `sol`에서 변수 제거 규칙을 구현하는 부분의 `eqns`, `tsubs`, `t2i`, `eqns'`, `tsubs'`는
각각 변수 제거 규칙에 나타나는 $P$, $S$, $\{t_i{\mapsto}\tau\}$, $\{t_i{\mapsto}\tau\}(P)$, $\{t_i{\mapsto}\tau\}(S)$에 해당한다. 

의미구조를 구현하는 `ev` 함수는 \ref{chap:TyInf}장의 코드를 그대로 가져오면 된다.
`eval'`은 이제 `infer`의 결과만으로는 알맞은 타입이 있는지 판단할 수 없으므로
`solve`로 연립방정식의 해가 존재하는지도 확인한다. 이렇게 `infer`와 `solve`가
성공한 경우에만 아래의 인터프리터 함수 `ev`를 실행한다고 가정한다.

In [25]:
ev :: Env -> Expr -> Val  -- infer와 solve가 성공한 경우만 실행 가정
ev _ _ = undefined -- 이전 장의 내용을 그대로 가져올 것

In [26]:
-- infer에 성공한 닫힌식에 대한 인터프리터
eval' e = [ev [] e | _ <- solve eqns]  where (t, eqns) = infer' e

In [27]:
e4 = Rec "f" (Just $ Arr Z Z) .
         Lam "i" (Just Z) $ If i (i `Add` App f (i `Add` Lit (-1))) i
    where f = Var "f"
          i = Var "i"
e4 -- rec f:Z->Z (\i:Z.if i t hen i + f(i + -1) else i)

\noindent
앞서 `infer`, `solve`, `ev`함수의 `undefined` 부분을 잘 채워넣었다면 다음의 셀이 에러 없이
`e4`를 활용한 0부터 100까지의 합 5050를 정상적으로 계산할 것이다.

In [28]:
-- rec f (\i.if i then i + f(i + -1) else i) 100
eval' (e4 `App` Lit 100) -- 100부터 0까지의 총합

: 

\newpage
\section*{요점정리}
* 이 장에서는 타입변수를 도입함으로써, 타입복원 및 타입유추 함수에서 모든 문제를 해결하려
  하지 말고 알맞은 타입이 존재하려면 타입변수라는 미지의 타입 정보가 어떤 조건을 만족해야
  하는지를 표현하는 방정식을 만들어내록 한다. 그리고, 그 방정식을 어떻게 잘 풀지는 별도의
  문제로 남겨두는 방식으로 접근한다.
* 타입변수가 만족해야 할 조건은 일치화(unification) 문제에 해당하며,
  타입복원과 타입유추 과정에서는 일반적으로 여러 개의 타입변수에 대한
  여러 개의 일치화 방정식을 동시에 만족해야 하는 일치화 연립방정식이 만들어진다.
* 일치화 방정식의 해는 유한한 개수의 타입변수를 타입으로 대응시키는 타입 치환이다.
  타입 치환을 방정식의 양쪽 혹은 어느 한쪽에만 적용했을 때 양쪽의 항이 같아진다면
  그 타입 치환이 바로 일치화 방정식의 해가 될 수 있다.
* 일치화 방정식의 해로 말이 되는 타입 치환은 1회 이상 몇 번을 반복 적용하든
  같은 결과를 유지하는 멱등성(idempotence)을 지녀야 한다. 특히, 치환 대상이
  되는 변수가 치환의 결과로 대응되는 타입에 나타날 경우 멱등성을 만족할 수 없다.
* 일치화 연립방정식의 풀이를 기술하는 규칙 기반 풀이 방식 $\mathcal{U}$는
  교집합을 베제한 합집합($\uplus$)으로 표현되어 있으므로 원론적으로는
  연립방정식 집합에서 방정식을 아무 순서로 하나씩 꺼내 가며 적용 가능한
  어떤 규칙이든 적용해 나가는 비결정적인 작은걸음 규칙이다.
* `[24]`번 셀의 하스켈 프로그램에서는 연립방정식을 리스트로 구현하여
  앞에서부터 나타나는 방정식을 순차적으로 풀이해 나간다.
  책의 본문에서 언급하지는 않았지만 풀이 방식 $\mathcal{U}$의
  작은걸음 규칙이 합류성을 만족하기 때문에 이렇게 하나의 정해진 순서로
  방정식의 풀이를 진행하더라도 어떤 다른 순서로 진행한 연립방정식의 풀이와도
  동등한 결과가 나올 것이라고 기대할 수 있다.

\section*{연습문제}
1. `[8]`번 셀의 `infer`에서 `undefined`로 남겨진 부분을 완성하여
   `Var`, `Lam`, `Add` 외의 다른 문법요소도 포함된 식에 대한 타입유추도 잘 동작하는지
   `infer` 또는 `infer'` 함수로 테스트해 보라.
1. 이 장에서 제공되거나 완성한 `recon`, `infer`, `solve` 함수로 아래 `[29]`번 셀과 같이
   재귀함 수 자신을 참조하는 `f`와 파라미터 `i`의 타입 표기가 모두 생략된 `e4'`의
   타입복원, 타입유추, 방정식 풀이의 각 단계가 잘 동작하는지 테스트하고,
   `eval'`로 `e4'`를 활용해 0부터 100까지 자연수의 합을 구하는 프로그램을 실행해 보라.
1. 일치화 연립방정식의 규칙 기반 풀이 방식 $\mathcal{U}$(그림 \ref{fig:RuleBasedUforTy})을 따라
   만들어지는 타입 치환은 멱등성(idempotence)이 보장된다. 어떻게 이러한 성질이 보장되는지 변수 제거 규칙을 중심으로 설명해 보라.

In [29]:
e4' = Rec "f" Nothing .
         Lam "i" Nothing $ If i (i `Add` App f (i `Add` Lit (-1))) i
    where f = Var "f"
          i = Var "i"
e4' -- rec f (\i.if i t then i + f(i + -1) else i)

\newpage
\section*{탐구과제}
1. 일치화(unification) 방정식 풀이를 내장한 Prolog는 대표적인 논리 프로그래밍(logic programming) 언어다.
   논리 프로그래밍 패러다임과 Prolog에 대해 알아보라.
   그리고 이 장에서 `infer`의 결과로 나온 일치화 연립방정식을 Prolog 언어로 옮겨보고
   실제로 Prolog가 방정식을 풀어주는지 확인해 보라.
1. 이 책에서는 단순타입에 대한 타입유추만을 다루고 있다.
   \ref{chap:ParaPoly}장에서 다룬 다형타입시스템을 지원하는 Haskell과 같은 언어에서는 타입유추가 어떻게 이루어지는지,
   Hindley--Milner (혹은 Hindley--Dammas--Milner) 타입유추 또는 let-polymorphism이라고도 불리는 타입유추 방식에 대해 알아보라.
1. 단순타입 언어에서는 $\textbf{let}$식을 문법설탕(syntactic sugar)으로 취급할 수 있다고 \ref{chap:TyInf}장에서 설명한 바 있다.
   그런데, Hindley--Milner 타입유추를 기반으로 하는 언어에서는,
   의미구조의 관점에서만 보면 $\textbf{let}$식이 단순히 문법설탕일 수 있지만
   타입규칙에서는 $\textbf{let}$식은 문법설탕 이상의 기능을 한다.
   어떤 기능을 하는지 알아보라.